<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [1]:
%load_ext sql

And then we import our movie database

In [2]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [3]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [5]:
%%sql 
SELECT *
FROM movies


 * sqlite:///movies.sqlite
Done.


title,production_year,IMDB_key,run_time
Licorice Pizza,2021,tt11271038,133
Killers of the Flower Moon,2022,tt5537002,120
The French Dispatch,2021,tt8847712,107


+ Show the performance dates for one of the movies.

In [6]:
%%sql
SELECT date
FROM performances
WHERE title = 'Licorice Pizza'


 * sqlite:///movies.sqlite
Done.


date
2022-03-01
2022-03-02
2022-03-03


+ Show all data concerning performances at a given theatere
  on a given date.

In [7]:
%%sql 
SELECT *
FROM performances
WHERE theater_name = 'Kino'


 * sqlite:///movies.sqlite
Done.


time,date,title,production_year,theater_name
19:00,2022-03-01,Licorice Pizza,2021,Kino
19:00,2022-03-02,Licorice Pizza,2021,Kino
19:00,2022-03-03,Licorice Pizza,2021,Kino


+ List all customers

In [8]:
%%sql
SELECT *
FROM customers


 * sqlite:///movies.sqlite
Done.


user_name,first_name,last_name,pass
alice123,Alice,Aliceson,password12
bob123,Bob,Bobson,password12


+ List all tickets

In [9]:
%%sql 
SELECT *
FROM tickets


 * sqlite:///movies.sqlite
Done.


id,theater_name,time,date,user_name


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [10]:
%%sql
INSERT
INTO    tickets(user_name, theater_name, time, date)
VALUES  ('alice123', 'Kino', '19:00', '2022-03-02');


 * sqlite:///movies.sqlite
1 rows affected.


[]

In newer versions of SQLite (since version 3.35, released
  in March 2021), and in
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html),
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


which would return the generated `s_id` for the new
  student.

  If your SQLite version is older than 3.35, and you can't
  upgrade, you can instead use the following idea: each row
  in a SQLite3 table has a `rowid` attribute, it is a unique
  integer which essentially tells in which order the rows
  were inserted, and it's not displayed in queries unless we
  ask for it. SQLite3 also have a function,
  `last_insert_rowid()`, which returns the `rowid` of the
  last inserted row of a table, so we can see the `s_id` of
  the most recently inserted student with the following
  query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [11]:
%%sql 
SELECT *
FROM tickets
WHERE user_name = 'alice123'


 * sqlite:///movies.sqlite
Done.


id,theater_name,time,date,user_name
0e59060ea99419755f9b4794791d9836,Kino,19:00,2022-03-02,alice123


+ Try to insert two movie theaters with the same name (this
  should fail).

In [12]:
%%sql
INSERT
INTO    theaters(theater_name, capacity)
VALUES  ('Malmö Biograf', 100),
        ('Malmö Biograf', 200); 

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: theaters.theater_name
[SQL: INSERT INTO theaters(theater_name, capacity)
VALUES  ('Malmö Biograf', 100),
        ('Malmö Biograf', 200);]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

+ Try to insert a performance where the theater doesnâ€™t
  exist in the database (this should fail).

In [17]:
%%sql
INSERT 
INTO    performances(time, date, title, production_year, theater_name)
VALUES ('12:00', '2022-01-02', 'The French Dispatch', 2021, 'SF Helsingborg');

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO performances(time, date, title, production_year, theater_name)
VALUES ('12:00', '2022-01-02', 'The French Dispatch', 2021, 'SF Helsingborg');]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

+ Create a ticket where either the user or the performance
  doesnâ€™t exist (this should fail).

In [18]:
%%sql
INSERT
INTO tickets(theater_name, time, date, user_name)
VALUES ('Kino', '19:00', '2022-03-01', 'unknownUserName')

 * sqlite:///movies.sqlite


IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO tickets(theater_name, time, date, user_name)
VALUES ('Kino', '19:00', '2022-03-01', 'unknownUserName')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)